In [1]:
import os
import sys

In [2]:
sys.path.append('../lib')
sys.path.append('../lib/sigkernel')

In [3]:
import cov_independence_test
import hsic_independence_test
import gen_processes

In [4]:
from importlib import reload
import pickle

In [5]:
reload(cov_independence_test);
reload(hsic_independence_test);
reload(gen_processes);

In [6]:
import datetime
import logging
import logging.handlers
LOGGER = logging.getLogger('logger')

def configure_logger():
    warning_handler = logging.StreamHandler()
    warning_handler.setLevel('WARNING')

    info_handler = logging.StreamHandler(sys.stdout)
    info_handler.setLevel('DEBUG')
    info_handler.addFilter(lambda record: record.levelno <= logging.INFO)

    LOGGER.addHandler(warning_handler)
    LOGGER.addHandler(info_handler)

def add_file_handlers(warning_log, info_log, debug_log):
    warning_log_handler = logging.FileHandler(warning_log)
    warning_log_handler.setLevel('WARNING')

    info_log_handler = logging.FileHandler(info_log)
    info_log_handler.addFilter(lambda record: record.levelno <= logging.INFO)
    info_log_handler.setLevel('INFO')

    debug_log_handler = logging.FileHandler(debug_log)
    debug_log_handler.addFilter(lambda record: record.levelno <= logging.DEBUG)
    debug_log_handler.setLevel('DEBUG')

    LOGGER.addHandler(warning_log_handler)
    LOGGER.addHandler(info_log_handler)
    LOGGER.addHandler(debug_log_handler)
    
OUTPUT_DIR = 'grand_hsic_test_outputs/{}/'.format(datetime.datetime.now().isoformat())
DEBUG_LOG_PATH = OUTPUT_DIR + 'debug_log'
INFO_LOG_PATH = OUTPUT_DIR + 'info_log'
WARNING_LOG_PATH = OUTPUT_DIR + 'warning_log'

configure_logger()
os.makedirs(OUTPUT_DIR)
add_file_handlers(WARNING_LOG_PATH, INFO_LOG_PATH, DEBUG_LOG_PATH)

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa.ar_model as ar
import torch
torch.set_default_dtype(torch.double)
import sigkernel_scale
import sigkernel
import warnings

In [8]:
import pprint
pp = pprint.PrettyPrinter(indent = 4)

In [9]:
import scipy.stats as stats

In [10]:
from tqdm import tqdm

In [11]:
from itertools import product

In [12]:
import matplotlib.pyplot as plt

In [13]:
from scipy.optimize import brentq

In [14]:
#SIG_KERNEL_SAMPLES_NO = 100
#SAMPLE_LENGTHS = [3, 6, 11, 31, 101]
#M_RUNS = 50
#CORRELATIONS = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.8]
#EXTINCTION_RATES = [0, 0.5, 1.]
#RBF_SIGMAS = [0.5, 1., 4., 6., 8.]
#DYADIC_ORDERS = [0, 2, 4]
#A=[0., 0.2, 0.5, 0.8]

SIG_KERNEL_SAMPLES_NO = 5
SAMPLE_LENGTHS = [3]
M_RUNS = 2
CORRELATIONS = [0, 0.1]
EXTINCTION_RATES = [0, 0.5]
RBF_SIGMAS = [0.5]
DYADIC_ORDERS = [0]
A=[0.]

In [15]:
KERNELS_NORMALIZED = [ (sigkernel_scale.LinearKernel(), None) ] 
KERNELS_NORMALIZED += [ (sigkernel_scale.RBFKernel(sigma), sigma) for sigma in RBF_SIGMAS ]
KERNELS_UNNORMALIZED = [ (sigkernel.LinearKernel(), None) ] 
KERNELS_UNNORMALIZED += [ (sigkernel.RBFKernel(sigma), sigma) for sigma in RBF_SIGMAS ]

In [16]:
LOGGER.setLevel('DEBUG')

In [17]:
def psi(x, M=4, a=1):
    x = x ** 2

    if x <= M:
        return x

    return M + M ** (1 + a) * (M ** (-a) - x ** (-a)) / a

In [18]:
def get_postscale_for_batch(paths, kernel):
    postscale = torch.zeros(paths.shape[0])
    for i in range(paths.shape[0]):
        kernel_xx = kernel.compute_kernel(paths[i:i+1, :, :], paths[i:i+1, :, :])
        target = psi(np.sqrt(kernel_xx.item()))
        f = lambda lam: target - kernel.compute_kernel(paths[i:i+1, :, :], paths[i:i+1, :, :], torch.tensor([lam]), torch.tensor([lam])).item()
        lam = min(max(brentq(f, 0., 1.1, xtol=1e-300, maxiter = 1000), 0), 1)
        postscale[i] = lam
    return postscale

In [19]:
def get_postscale_for_batch_adaptive_dyadic_order(paths, static_kernel):
    LOGGER.debug('FINDING POSTSCALE FOR BATCH')
    
    dyadic_order = 0
    warnings.filterwarnings("error", category = RuntimeWarning)

    while True:
        try: 
            postscale = get_postscale_for_batch(paths, sigkernel_scale.SigKernel(static_kernel, dyadic_order));
            break
        except (ValueError, RuntimeWarning) as e:
            LOGGER.debug('Failed, increasing dyadic order')
            dyadic_order += 1

    warnings.resetwarnings()
    LOGGER.debug('Found postscale: ' + str(postscale))
    LOGGER.debug('Used dyadic order: ' + str(dyadic_order))
    
    return postscale, dyadic_order

In [20]:
def run_experiment_for_unnormalized_kernels(proc_gen_fct):
    results = []
    
    for kernel, kernel_param in KERNELS_UNNORMALIZED:
        for run in range(M_RUNS):
            for order in DYADIC_ORDERS:
                LOGGER.info('Experiment for unnormalized kernel: run {}, order {}, kernel param {}'.format(run, order, kernel_param))
                
                processes = proc_gen_fct()
                X_raw = processes[:, :, 0]
                Y_raw = processes[:, :, 1]
                X_lead_lag_chopped = torch.stack((X_raw[:, 1:], X_raw[:, :-1]), axis = 2)
                Y_lead_lag_chopped = torch.stack((Y_raw[:, 1:], Y_raw[:, :-1]), axis = 2)

                null, hsic_score, p_val = hsic_independence_test.get_test_result_matrix_shuffle(X_lead_lag_chopped, Y_lead_lag_chopped, static_kernel = kernel, dyadic_order = order)
                
                LOGGER.info('Result p_val: {}'.format(p_val))
                
                results.append({
                    'kernel_type' : 'unnormalized',
                    'kernel_param' : kernel_param,
                    'dyadic_order' : order,
                    'processes' : processes,
                    'null_approx' : null,
                    'hsic_score' : hsic_score,
                    'p_val' : p_val,
                    'postscale_x': None,
                    'postscale_x_do': None,
                    'postscale_y': None,
                    'postscale_y_do': None,
                })
    
    return results

In [21]:
def run_experiment_for_normalized_kernels(proc_gen_fct):
    results = []
    
    for kernel, kernel_param in KERNELS_NORMALIZED:
        for run in range(M_RUNS):
            for order in DYADIC_ORDERS:
                LOGGER.info('Experiment for normalized kernel: run {}, order {}, kernel param {}'.format(run, order, kernel_param))
                
                processes = proc_gen_fct()
                X_raw = processes[:, :, 0]
                Y_raw = processes[:, :, 1]
                X_lead_lag_chopped = torch.stack((X_raw[:, 1:], X_raw[:, :-1]), axis = 2)
                Y_lead_lag_chopped = torch.stack((Y_raw[:, 1:], Y_raw[:, :-1]), axis = 2)

                postscale_x, order_x = get_postscale_for_batch_adaptive_dyadic_order(X_lead_lag_chopped, kernel);
                postscale_y, order_y = get_postscale_for_batch_adaptive_dyadic_order(Y_lead_lag_chopped, kernel);

                null, hsic_score, p_val = hsic_independence_test.get_test_result_matrix_shuffle(X_lead_lag_chopped, Y_lead_lag_chopped, static_kernel = kernel, dyadic_order = order, postscale_x = postscale_x, postscale_y = postscale_y)
                
                LOGGER.info('Result p_val: {}'.format(p_val))
                
                results.append({
                    'kernel_type' : 'normalized',
                    'kernel_param' : kernel_param,
                    'dyadic_order' : order,
                    'processes' : processes,
                    'null_approx' : null,
                    'hsic_score' : hsic_score,
                    'p_val' : p_val,
                    'postscale_x': postscale_x,
                    'postscale_x_do': order_x,
                    'postscale_y': postscale_y,
                    'postscale_y_do': order_y,
                })
    
    return results

In [22]:
def push_results(results):
    LOGGER.debug('Pushing results')
    LOGGER.debug('Current results on stack: ' + str(push_results.paths))
    if len(push_results.paths) == 5: 
        oldest_results_file = push_results.paths.pop(0)
        os.remove(oldest_results_file)
        
    output_file_path = OUTPUT_DIR + 'intermediate_results_{}.pickle'.format(datetime.datetime.now().isoformat()).replace(' ', '_').replace(':', '.')        
    
    with open(output_file_path, 'wb') as f:
        pickle.dump(results, f)
    
    LOGGER.debug('Saved ' + output_file_path)
    push_results.paths.append(output_file_path)

push_results.paths = []

In [23]:
results = []
    #(lambda sl, a, param: gen_processes.gen_ar_iid_samples_extinct_gaussian_burn_in(sl, SIG_KERNEL_SAMPLES_NO, a=a, extinction_rate=param), 'extinct'),
    #(lambda sl, a, param: gen_processes.gen_ar_iid_samples_burn_in(sl, SIG_KERNEL_SAMPLES_NO, a=a, extinction_rate=param), 'corr'),
for sl in SAMPLE_LENGTHS:
    for a in A:
        for ext_rate in EXTINCTION_RATES:
            LOGGER.info('============ STARTING ANOTHER RUN : EXTINCT GAUSSIAN ==============')
            LOGGER.info('Sample length: {}, A: {}, extinction rate: {}'.format(sl, a, ext_rate))

            proc_gen_fct = lambda: gen_processes.gen_ar_iid_samples_extinct_gaussian_burn_in(sl, SIG_KERNEL_SAMPLES_NO, a=a, extinction_rate=ext_rate)
            results_norm = run_experiment_for_normalized_kernels(proc_gen_fct)
            results_unnorm = run_experiment_for_unnormalized_kernels(proc_gen_fct)
            
            for result in results_norm + results_unnorm:
                result['process_type'] = 'extinct_gaussian'
                result['a'] = a
                result['process_param'] = ext_rate

            results += results_norm + results_unnorm
            
            push_results(results)
            LOGGER.debug('============ FINISHED ANOTHER RUN : EXTINCT GAUSSIAN ==============')
        for corr in CORRELATIONS:
            LOGGER.debug('============ STARTING ANOTHER RUN : LINEAR CORRELATION ==============')
            LOGGER.info('Sample length: {}, A: {}, correlation: {}'.format(sl, a, corr))

            proc_gen_fct = lambda: gen_processes.gen_ar_iid_samples_burn_in(sl, SIG_KERNEL_SAMPLES_NO, a=a, corr=corr)
            results_norm = run_experiment_for_normalized_kernels(proc_gen_fct)
            results_unnorm = run_experiment_for_unnormalized_kernels(proc_gen_fct)
            
            for result in results_norm + results_unnorm:
                result['process_type'] = 'lin_corr'
                result['a'] = a
                result['process_param'] = ext_rate

            results += results_norm + results_unnorm
            
            push_results(results)
            LOGGER.debug('============ FINISHED ANOTHER RUN : LINEAR CORRELATION ==============')

        

============ STARTING ANOTHER RUN : EXTINCT GAUSSIAN ==============
Sample length: 3, A: 0.0, extinction rate: 0
Experiment for normalized kernel: run 0, order 0, kernel param None
FINDING POSTSCALE FOR BATCH
Found postscale: tensor([1.0000, 1.0000, 0.8661, 0.6857, 0.9901])
Used dyadic order: 0
FINDING POSTSCALE FOR BATCH
Found postscale: tensor([1.0000, 1.0000, 0.9995, 0.4115, 1.0000])
Used dyadic order: 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 1267.12it/s]

Result p_val: 0.36
Experiment for normalized kernel: run 1, order 0, kernel param None


FINDING POSTSCALE FOR BATCH
Found postscale: tensor([1.0000, 1.0000, 1.0000, 1.0000, 0.3622])
Used dyadic order: 0
FINDING POSTSCALE FOR BATCH
Found postscale: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])
Used dyadic order: 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 1065.60it/s]

...

Result p_val: 0.02
Experiment for unnormalized kernel: run 0, order 0, kernel param 0.5



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 3128.63it/s]

Result p_val: 0.34
Experiment for unnormalized kernel: run 1, order 0, kernel param 0.5



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 3261.26it/s]

Result p_val: 0.7
Pushing results
Current results on stack: ['grand_hsic_test_outputs/2022-04-08T13:52:06.349957/intermediate_results_2022-04-08T13.52.10.509814.pickle', 'grand_hsic_test_outputs/2022-04-08T13:52:06.349957/intermediate_results_2022-04-08T13.52.12.122211.pickle', 'grand_hsic_test_outputs/2022-04-08T13:52:06.349957/intermediate_results_2022-04-08T13.52.13.822068.pickle']
Saved grand_hsic_test_outputs/2022-04-08T13:52:06.349957/intermediate_results_2022-04-08T13.52.15.091865.pickle
============ FINISHED ANOTHER RUN : LINEAR CORRELATION ==============


In [24]:
results_final = {
    'results' : results,
    'hyperparams' : {
        'SIG_KERNEL_SAMPLES_NO' : SIG_KERNEL_SAMPLES_NO,
        'SAMPLE_LENGTHS' : SAMPLE_LENGTHS,
        'M_RUNS' : M_RUNS,
        'CORRELATIONS' : CORRELATIONS,
        'EXTINCTION_RATES' : EXTINCTION_RATES,
        'RBF_SIGMAS' : RBF_SIGMAS,
        'DYADIC_ORDERS' : DYADIC_ORDERS,
        'A': A
    }
}
output_file_path = OUTPUT_DIR + 'results_final.pickle'
with open(output_file_path, 'wb') as f:
    pickle.dump(results_final, f)


In [25]:
pp.pprint(results_final)

{   'hyperparams': {   'A': [0.0],
                       'CORRELATIONS': [0, 0.1],
                       'DYADIC_ORDERS': [0],
                       'EXTINCTION_RATES': [0, 0.5],
                       'M_RUNS': 2,
                       'RBF_SIGMAS': [0.5],
                       'SAMPLE_LENGTHS': [3],
                       'SIG_KERNEL_SAMPLES_NO': 5},
    'results': [   {   'a': 0.0,
                       'dyadic_order': 0,
                       'hsic_score': 0.8477284382893471,
                       'kernel_param': None,
                       'kernel_type': 'normalized',
                       'null_approx': tensor([0.3066, 0.3066, 0.3164, 0.3164, 0.3193, 0.3193, 0.3338, 0.3752, 0.3839,
        0.4462, 0.5062, 0.5080, 0.5911, 0.5911, 0.5968, 0.5968, 0.6017, 0.6082,
        0.6335, 0.6349, 0.6349, 0.6471, 0.6471, 0.6695, 0.6695, 0.6888, 0.7056,
        0.7221, 0.7317, 0.8008, 0.8381, 0.8419, 0.8716, 0.9316, 0.9316, 0.9349,
        0.9349, 0.9434, 0.9559, 1.0512, 1.0977, 1.144

                       'processes': tensor([[[-0.8207,  1.1484],
         [-0.2085, -1.7583],
         [ 0.8793, -1.0705]],

        [[ 0.9691,  0.6674],
         [ 0.1401,  1.4712],
         [-1.5613, -0.9394]],

        [[ 0.5698,  1.1188],
         [-0.0632, -0.9440],
         [-0.3704, -1.1602]],

        [[-0.3324, -0.5064],
         [ 1.0107, -0.6980],
         [-1.1945, -0.2469]],

        [[-0.7465, -0.0771],
         [-1.7239,  0.0555],
         [-1.2382,  0.8195]]])},
                   {   'a': 0.0,
                       'dyadic_order': 0,
                       'hsic_score': 0.812280911884768,
                       'kernel_param': 0.5,
                       'kernel_type': 'normalized',
                       'null_approx': tensor([0.7999, 0.8094, 0.8106, 0.8106, 0.8119, 0.8125, 0.8134, 0.8168, 0.8191,
        0.8196, 0.8206, 0.8206, 0.8243, 0.8245, 0.8294, 0.8294, 0.8327, 0.8340,
        0.8348, 0.8370, 0.8370, 0.8370, 0.8396, 0.8419, 0.8491, 0.8514, 0.8514,
        0.85